In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Development Kit - Sequential Agents

### Install adk

In [1]:
%pip install --upgrade --quiet google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00


In [2]:
# @title Project Information

PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"

MODEL_LIST = ["gemini-2.0-flash", \
              "gemini-2.5-pro-preview-03-25", \
              "gemini-2.5-flash-preview-04-17"]

MODEL_NAME = MODEL_LIST[2]
print(MODEL_NAME)

gemini-2.5-flash-preview-04-17


In [3]:
# @title Authentication to access to GCP
import sys
from pprint import pprint

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initialize Vertex AI

* Please refer to configuration to **initialize Vertex AI : [Link](https://google.github.io/adk-docs/get-started/quickstart/#set-up-the-model)**
* You can set up api key rather than usning Vertex AI.


In [4]:
import os

GOOGLE_GENAI_USE_VERTEXAI = "TRUE"                            #@param {type:"string"}
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = GOOGLE_GENAI_USE_VERTEXAI

GOOGLE_CLOUD_PROJECT = "ai-hangsik"                           #@param {type:"string"}
os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT

GOOGLE_CLOUD_LOCATION = "us-central1"                         #@param {type:"string"}
os.environ["GOOGLE_CLOUD_LOCATION"] = GOOGLE_CLOUD_LOCATION




## adk with tools

In [6]:
import asyncio

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools import google_search
from google.adk.tools import ToolContext

from google.adk.agents import SequentialAgent
from google.adk.agents import ParallelAgent
from google.adk.agents import LoopAgent


In [10]:
# --- Tool Definition ---
def exit_loop(tool_context: ToolContext):
  """Call this function ONLY when the critique indicates no further changes are needed, signaling the iterative process should end."""
  print(f"  [Tool Call] exit_loop triggered by {tool_context.agent_name}")
  tool_context.actions.escalate = True
  # Return empty dict as tools should typically return JSON-serializable output
  return {}

In [18]:
STATE_INITIAL_TOPIC = "initial_sentence"
STATE_CURRENT_DOC = "translated_sentence"
STATE_CRITICISM = "criticism"
COMPLETION_PHRASE = "No major issues found."

translation_agent = LlmAgent(
    name="TranslationAgent",
    model=MODEL_NAME,
    instruction=f"""You are a helpful AI translater.
                   Add the title "## Translate Sentence" to the response.

                   Sentence: {STATE_INITIAL_TOPIC}

    """,
    description="Translation Agent",
    output_key=STATE_CURRENT_DOC,
)

critic_agent_in_loop = LlmAgent(
    name="CriticAgent",
    model=MODEL_NAME,
    instruction=f"""You are a constructive critic AI reviewing the translated sentence.
                       Add the title "## Criticise the translated sentence" to the response.

                    **Document to Review:**
                    ```
                    {STATE_CURRENT_DOC}
                    ```

                    **Task:**
                    Review the translated sentence for clarity.

                    IF you identify 1-2 *clear and actionable* ways the translation could be improved to better.
                    Provide these specific suggestions concisely. Output *only* the critique text.

                    ELSE IF the translation is ok:
                    Respond *exactly* with the phrase "{COMPLETION_PHRASE}" and nothing else.
                    Do not add explanations. Output only the critique OR the exact completion phrase.

    """,
    description="Critic Agent",
    output_key=STATE_CRITICISM,
)

refiner_agent_in_loop = LlmAgent(
    name="RefinerAgent",
    model=MODEL_NAME,
    include_contents='none',

    instruction=f"""You are a translation Assistant refining the translated sentence.
                    Add the title "## Refining process for translated Sentence" to the response.

    **Current Document:**
    ```
    {STATE_CURRENT_DOC}
    ```
    **Critique/Suggestions:**
    {STATE_CRITICISM}

    **Task:**
    Analyze the 'Critique/Suggestions'.

    IF the critique is *exactly* "{COMPLETION_PHRASE}":
      You MUST call the 'exit_loop' function. Do not output any text.
    ELSE (the critique contains actionable feedback):
      Carefully apply the suggestions to improve the 'Current Document'. Output *only* the refined document text.

    Do not add explanations. Either output the refined document OR call the exit_loop function. """,

    description="Refines the document based on critique, or calls exit_loop if critique indicates completion.",
    tools=[exit_loop],
    output_key=STATE_CURRENT_DOC
)

refinement_loop = LoopAgent(
    name="RefinementLoop",
    # Agent order is crucial: Critique first, then Refine/Exit
    sub_agents=[
        critic_agent_in_loop,
        refiner_agent_in_loop,
    ],
    max_iterations=2
)

# STEP 3: Overall Sequential Pipeline
# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = SequentialAgent(
    name="IterativeWritingPipeline",
    sub_agents=[
        translation_agent, # Run first to create initial doc
        refinement_loop       # Then run the critique/refine loop
    ],
    description="Writes an initial document and then iteratively refines it with critique using an exit tool."
)



In [19]:
app_name = "Assistant"
user_id = "shins"

query = """You want to build an agent that can generate images of food, but sometimes when you want to generate a specific number of items (e.g. 5 bananas), it generates a different number of those items in the image (e.g. an image of 7 bananas). You have two tools: generate_image, count_food_items. Because you want to keep generating images until it either correctly generates the specified number of items, or after a certain number of iterations, you should build your agent using a LoopAgent. 를 한국어로 번역해주세요."""


session_service = InMemorySessionService()
session = session_service.create_session(app_name=app_name, user_id=user_id)

runner = Runner(agent=root_agent,
                app_name=session.app_name,
                session_service=session_service)

content = types.Content(role='user', parts=[types.Part(text=query)])

events = runner.run(user_id=session.user_id,
                    session_id=session.id,
                    new_message=content)

for event in events:
    print(event.content.parts[0].text)

    # if event.is_final_response():
    #     final_response = event.content.parts[0].text
    #     print(f"Agent Response: {final_response}\n")
    # else:
    #     print("No results")

## Translate Sentence

음식 이미지를 생성할 수 있는 에이전트를 구축하려고 하지만, 특정 개수의 항목(예: 바나나 5개)을 생성하고 싶을 때 이미지에 다른 개수(예: 바나나 7개 이미지)를 생성하는 경우가 있습니다. 에이전트에게는 `generate_image`와 `count_food_items`라는 두 가지 도구가 있습니다. 에이전트가 지정된 개수를 정확히 생성하거나 일정 횟수의 반복을 수행할 때까지 이미지를 계속 생성하고 싶기 때문에 LoopAgent를 사용하여 에이전트를 구축해야 합니다.
No major issues found.


  [Tool Call] exit_loop triggered by RefinerAgent
None
None


## End of notebook